In [ ]:
# --- COLAB SETUP (run this first) ---
import os, sys

REPO_URL = "https://github.com/mrc-rossoni/surface-book"
REPO_DIR = "surface-book"

if not os.path.exists(REPO_DIR):
    !git clone --depth 1 {REPO_URL}

sys.path.append(REPO_DIR)
!pip install -q -r {REPO_DIR}/requirements.txt
print("✅ Setup complete")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from code.curves import bezier_eval, de_casteljau

In [ ]:
P = np.array([
    [0.0, 0.0],
    [0.2, 1.2],
    [0.8, 1.0],
    [1.0, 0.0]
])


In [ ]:
u = np.linspace(0, 1, 200)
C = bezier_eval(P, u)

plt.figure()
plt.plot(P[:,0], P[:,1], "--o", label="Control polygon")
plt.plot(C[:,0], C[:,1], label="Bezier curve")
plt.axis("equal")
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
u_test = 0.35
c1 = bezier_eval(P, np.array([u_test]))[0]
c2 = de_casteljau(P, u_test)

print("Bernstein:", c1)
print("De Casteljau:", c2)
print("Difference:", np.linalg.norm(c1 - c2))


Exercises section (markdown cell)

Change control points to create inflection points

Increase degree with random control points and test stability

Plot curve + convex hull polygon